Cleaning the dataset is the first part of the Tardis project. 
We have to select which values are mandatory, which must be corrected and which must be ignored.

In [205]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The clean_average function handles the average column. It returns a new dataframe with the column specified according to the following rules
    - N/A or <null> value -> 0.0
    - Values formated to HOUR/MIN/SEC (as a string)

In [206]:
def clean_average(column, dataframe):
    df = dataframe.copy()
    
    #possibly add verif for aberrant values
    df[column] = df[column].fillna(0.0)
    total_seconds = (df[column] * 60).astype(int)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    df[column] = hours.astype(str) + '.' + minutes.astype(str) + '.' + seconds.astype(str)
    return df

The clean_Pct function handles the Pct column. It returns a new dataframe with the column specified according to the following rules
    - Pct > 100 -> <null>
    - round values to 2 digits

In [207]:
def clean_Pct(column, dataframe):
    df = dataframe.copy()

    df[column] = df[column].where(df[column] < 100.0, np.nan)
    df[column] = df[column].round(2)
    return df

The handle_column_with_train_number handles the columns that contain a number of train. It returns a new dataframe with the column specified according to the following rules:
    - replace non numeric values or float values to null
    - check if the values is < to the number of train at departure (because it is not possible to have more delayed trains then trains at departure)

In [208]:
def handle_column_with_train_number(column, dataframe):
    df = dataframe.copy()

    df[column] = df[column].replace(['N/A', ''], np.nan)
    scheduled = pd.to_numeric(df['Number of scheduled trains'].replace(['N/A', ''], np.nan), errors='coerce')
    df[column] = df[column].where(df[column] <= scheduled, np.nan)
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df[column] = df[column].where(df[column].apply(lambda x: pd.isna(x) or x.is_integer()))
    df[column] = df[column].fillna(0).astype(int)
    return df


The tab containing all the destinations for spelling correction and the diff function to do it.

In [209]:
all_city = [
    "PARIS MONTPARNASSE",
    "QUIMPER",
    "ST MALO",
    "ST PIERRE DES CORPS",
    "STRASBOURG",
    "PARIS NORD",
    "LYON PART DIEU",
    "TOURCOING",
    "NANTES",
    "PARIS VAUGIRARD",
    "BORDEAUX ST JEAN",
    "PARIS LYON",
    "MARNE LA VALLEE",
    "CHAMBERY CHALLES LES EAUX",
    "MARSEILLE ST CHARLES",
    "FRANCFORT",
    "ZURICH",
    "ANGOULEME",
    "POITIERS",
    "TOURS",
    "METZ",
    "REIMS",
    "PARIS EST",
    "DOUAI",
    "MULHOUSE VILLE",
    "VALENCE ALIXAN TGV",
    "STUTTGART",
    "BARCELONA",
    "ANGERS SAINT LAUD",
    "LAVAL",
    "NANCY",
    "LILLE",
    "GRENOBLE",
    "LE CREUSOT MONTCEAU MONTCHANIN",
    "MACON LOCHE",
    "NIMES",
    "ITALIE",
    "RENNES",
    "BREST",
    "LA ROCHELLE VILLE",
    "LE MANS",
    "VANNES",
    "DUNKERQUE",
    "AVIGNON TGV",
    "BELLEGARDE (AIN)",
    "BESANCON FRANCHE COMTE TGV",
    "DIJON VILLE",
    "MONTPELLIER",
    "MADRID",
    "ARRAS",
    "AIX EN PROVENCE TGV",
    "ANNECY",
    "NICE VILLE",
    "SAINT ETIENNE CHATEAUCREUX",
    "TOULON",
    "GENEVE",
    "PERPIGNAN",
    "LAUSANNE",
    "TOULOUSE MATABIAU"
]

def diff(string, to_cmp):
    nb_error = 0
    for i in range(len(string)):
        if string[i] != to_cmp[i]:
            nb_error += 1
    return nb_error

We start by reading he csv file and removing duplicates values.

In [210]:
df = pd.read_csv('dataset.csv', delimiter=';', on_bad_lines='warn')
df.drop_duplicates(inplace=True, subset=['Date', 'Departure station', 'Arrival station'])
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

We clean the 'Date' column by applying a YYYY/MM format, correcting small mistakes and removing incoherent values like e.g. 2018-14 or 2142-03
We decided to not make the 'Date' mandatory information, so a null value doesn't remove the entire line.

In [211]:
df["Date"] = df["Date"].str.replace('[^0-9]', '', regex=True)
df["Date"] = df["Date"].apply(lambda x: str(x))

valid_format = df['Date'].str.match(r'^\d{6}$')
df['Date'] = df['Date'].where(valid_format, '--------')

df["Date"] = df["Date"].apply(lambda x: x[0:4] + '-' + x[4:6])
df["Date"] = df["Date"].str.replace('nan-', '')
df["Date"] = df["Date"].str.replace('-------', '')

df['Date'] = df['Date'].where(pd.to_numeric(df['Date'].str[-2:].astype('str')) <= 12, '')
df['Date'] = df['Date'].where(pd.to_numeric(df['Date'].str[:4].astype('str')) <= 2025, '')


We clean the 'Service' column by correcting the spelling mistakes
We decided to not make the 'Service' mandatory information, so a null value doesn't remove the entire line.

In [212]:
df["Service"] = df["Service"].apply(lambda x: str(x))
df["Service"] = df["Service"].str.replace('nan', '')
df['Service'] = df['Service'].where(df["Service"].str.len() != 8, 'National')
df['Service'] = df['Service'].where(df["Service"].str.len() != 13, 'International')
df['Service'] = df['Service'].where((df["Service"].str.len() == 13) | (df['Service'].str.len() == 8), '')

We clean the 'Departure station' and 'Arrival station' by correcting the spelling mistakes.
We decided to make the 'Departure station' and 'Arrival station' mandatory information, to allow us to locate and verify further informations.

In [213]:
df["Departure station"] = df["Departure station"].apply(lambda x: str(x))
df = df.drop(df[df["Departure station"] == 'nan'].index)
df["Arrival station"] = df["Arrival station"].apply(lambda x: str(x))
df = df.drop(df[df["Arrival station"] == 'nan'].index)

In [214]:
for j in range(len(df)):
    station = df.at[df.index[j], 'Departure station']
    
    for element in all_city:
        if len(station) != len(element):
            continue
        if diff(station, element) == 1:
            df.at[df.index[j], 'Departure station'] = 'non valid'
            break

df = df.drop(df[df["Departure station"] == 'non valid'].index)

We convert the columns that represents a number of train to int (because you cannot have a half train) and verify if the values are coherent with the number of trains at departure
We also remove the lines where they are no scheduled trains

In [215]:
df = handle_column_with_train_number('Number of scheduled trains', df)
df = df.drop(df[df["Number of scheduled trains"] == 0].index)
df = handle_column_with_train_number('Number of cancelled trains', df)
df = handle_column_with_train_number('Number of trains delayed at departure', df)
df = handle_column_with_train_number('Number of trains delayed at arrival', df)
df = handle_column_with_train_number('Number of trains delayed > 15min', df)
df = handle_column_with_train_number('Number of trains delayed > 30min', df)
df = handle_column_with_train_number('Number of trains delayed > 60min', df)

We apply the "clean_average" function on all the average columns.

In [216]:
df = clean_average('Average journey time', df)
df = clean_average('Average delay of late trains at departure', df)
df = clean_average('Average delay of late trains at arrival', df)
df = clean_average('Average delay of all trains at arrival', df)
df = clean_average('Average delay of trains > 15min (if competing with flights)', df)

We check if the percentages are coherent (nothing > 100%) and then we apply the "clean_Pct" function on all the Pct columns.
Percentages are only useful in visualization so we don't drop the entire line if the percentages are wrong.

In [217]:
# Reset index if necessary
df = df.reset_index(drop=True)

columns_to_check = [
    'Pct delay due to passenger handling (crowding, disabled persons, connections)',
    'Pct delay due to station management and equipment reuse',
    'Pct delay due to traffic management',
    'Pct delay due to rolling stock',
    'Pct delay due to infrastructure',
    'Pct delay due to external causes'
]

for i in range(len(df)):
    total_pct = sum(df[col][i] for col in columns_to_check)

    if not (99 <= total_pct <= 101):
        for col in columns_to_check:
            df.at[i, col] = np.nan

df = clean_Pct("Pct delay due to passenger handling (crowding, disabled persons, connections)", df)
df = clean_Pct('Pct delay due to station management and equipment reuse',df)
df = clean_Pct('Pct delay due to rolling stock', df)
df = clean_Pct('Pct delay due to traffic management', df)
df = clean_Pct('Pct delay due to infrastructure', df)
df = clean_Pct('Pct delay due to external causes', df)

Finally, we convert our cleaned dataframe into a new csv file

In [218]:
df.to_csv('cleaned_dataset.csv', index=False)

In [219]:
df

,Date,Service,Departure station,Arrival station,Average journey time,Number of scheduled trains,Number of cancelled trains,Cancellation comments,Number of trains delayed at departure,Average delay of late trains at departure,Average delay of all trains at departure,Departure delay comments,Number of trains delayed at arrival,Average delay of late trains at arrival,Average delay of all trains at arrival,Arrival delay comments,Number of trains delayed > 15min,Average delay of trains > 15min (if competing with flights),Number of trains delayed > 30min,Number of trains delayed > 60min,Pct delay due to external causes,Pct delay due to infrastructure,Pct delay due to traffic management,Pct delay due to rolling stock,Pct delay due to station management and equipment reuse,"Pct delay due to passenger handling (crowding, disabled persons, connections)"
0,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,2.45.0,222,0,NaN,8,0.2.52,0.10,NaN,34,0.21.31,0.5.41,NaN,22,0.5.41,5,0,15.38,30.77,38.46,11.54,3.85,0.00
1,2018-01,National,PARIS MONTPARNASSE,QUIMPER,3.40.0,248,1,NaN,37,0.9.30,NaN,NaN,0,0.55.41,0.7.34,"Ce mois-ci, l'OD a été touchée par les inciden...",26,0.7.32,17,7,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01,National,PARIS MONTPARNASSE,ST MALO,2.36.0,102,0,NaN,12,0.19.54,1.97,NaN,13,0.48.37,0.6.47,"Ce mois-ci, l'OD a été touchée par les inciden...",8,0.6.43,6,4,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,1.1.0,391,2,NaN,61,0.0.0,0.89,NaN,71,0.12.24,0.3.20,NaN,17,0.3.20,6,0,21.21,42.42,9.09,21.21,6.06,0.00
4,2018-01,National,QUIMPER,PARIS MONTPARNASSE,3.43.0,256,1,NaN,0,0.16.19,0.58,NaN,21,0.39.5,0.5.56,NaN,21,0.5.55,9,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,2024-12,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,1.4.0,539,10,NaN,49,0.12.14,0.75,NaN,63,0.0.0,0.2.43,NaN,22,0.27.55,5,1,6.56,16.39,29.51,29.51,11.48,6.56
7556,2024-12,National,PARIS MONTPARNASSE,VANNES,2.38.0,284,2,NaN,27,0.22.29,1.57,NaN,36,0.41.34,0.0.0,NaN,26,0.52.40,0,0,33.33,16.67,19.44,22.22,5.56,2.78
7557,2024-12,National,QUIMPER,PARIS MONTPARNASSE,3.51.0,274,1,NaN,0,0.0.0,1.52,NaN,39,0.56.9,0.9.14,NaN,39,0.56.9,0,13,41.03,20.51,7.69,17.95,7.69,5.13
7558,2024-12,National,SAINT ETIENNE CHATEAUCREUX,PARIS LYON,2.48.0,110,0,NaN,20,1.58.17,1.85,NaN,25,0.28.37,0.8.0,NaN,19,0.33.24,6,1,NaN,NaN,NaN,NaN,NaN,NaN
